<a href="https://colab.research.google.com/github/ArielFix/DLINtroProject/blob/NNModel/LSTM_Model_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Library for Splitting Dataset
from sklearn.model_selection import train_test_split

#Libraries for NN
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout

In [82]:
from google.colab import drive
drive.mount('/content/gdrive')
train_data_path = '/content/gdrive/MyDrive/IDL_Assignments/NLP_Project/data/train.csv'
test_data_path = '/content/gdrive/My Drive/IDL_Assignments/NLP_Project/data/test.csv'
submission_path = '/content/gdrive/MyDrive/IDL_Assignments/NLP_Project/data/sample_submission.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [83]:
train_data = pd.read_csv(train_data_path)

In [84]:
def toclean_text(text):

    
    clean_text = [char for char in text if char not in string.punctuation]
   
    clean_text = ''.join(clean_text)
    
        
    return clean_text

In [85]:
train_data['clean_text'] = train_data['text'].apply(toclean_text) #This line gives the text without punctuation

More pre-processing must be done! such as replace abbreviations with the original word, remove URLs, remove numbers, remove emojis..etc...

In [86]:
max_features=3000
tokenizer=Tokenizer(num_words=max_features,split=' ')
tokenizer.fit_on_texts(train_data['clean_text'].values) #Fit_on_texts() Updates internal vocabulary based on a list of texts. 
X = tokenizer.texts_to_sequences(train_data['clean_text'].values) #texts_to_sequences() Transforms each text in texts to a sequence of numerical value.
X = pad_sequences(X)

Model-

In [87]:
y = train_data['target']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =41) #Split arrays or matrices into random train and test subsets.


In [88]:
embed_dim = 32 
lstm_out = 32 #output_dim: Dimension of embedding vector
model = Sequential() #a plain stack of layers where each layer has exactly one input tensor and one output tensor.
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1])) #using embedding better then one-hot in the cases where mose indices are zero
model.add(Dropout(0.2)) #Dropout layer randomly sets input units to 0 with a frequency of rate at each step during training time, which helps prevent overfitting.
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.4))
model.add(Dense(1,activation='sigmoid'))
adam = optimizers.Adam(learning_rate=0.002)
model.compile(loss = 'binary_crossentropy', optimizer=adam ,metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 31, 32)            96000     
                                                                 
 dropout_3 (Dropout)         (None, 31, 32)            0         
                                                                 
 lstm_3 (LSTM)               (None, 32)                8320      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 104,353
Trainable params: 104,353
Non-trainable params: 0
_________________________________________________________________
None


In [89]:
model.fit(X_train, y_train, epochs = 10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
191/191 [==============================] - 10s 38ms/step - loss: 0.5456 - accuracy: 0.7202 - val_loss: 0.4320 - val_accuracy: 0.8109
Epoch 2/10
191/191 [==============================] - 7s 36ms/step - loss: 0.3818 - accuracy: 0.8379 - val_loss: 0.4394 - val_accuracy: 0.8037
Epoch 3/10
191/191 [==============================] - 7s 36ms/step - loss: 0.3346 - accuracy: 0.8627 - val_loss: 0.4494 - val_accuracy: 0.8017
Epoch 4/10
191/191 [==============================] - 7s 35ms/step - loss: 0.2987 - accuracy: 0.8819 - val_loss: 0.5052 - val_accuracy: 0.7820
Epoch 5/10
191/191 [==============================] - 7s 35ms/step - loss: 0.2658 - accuracy: 0.8966 - val_loss: 0.5204 - val_accuracy: 0.7827
Epoch 6/10
191/191 [==============================] - 7s 36ms/step - loss: 0.2352 - accuracy: 0.9033 - val_loss: 0.5668 - val_accuracy: 0.7814
Epoch 7/10
191/191 [==============================] - 7s 36ms/step - loss: 0.2117 - accuracy: 0.9133 - val_loss: 0.6304 - val_accuracy: 0.778

In [90]:
y_pred = model.predict(X_test).round()

48/48 [==============================] - 1s 5ms/step


In [91]:
y_pred

array([[0.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

Evaluation

In [92]:
train_accuracy = round(metrics.accuracy_score(y_train,model.predict(X_train).round())*100)
train_accuracy

191/191 [==============================] - 1s 5ms/step


96

In [93]:
print('Accuracy  is  : ', (metrics.accuracy_score(y_test, y_pred)))
print('Recall  is    : ', (metrics.recall_score(y_test, y_pred)))
print('Precision  is : ', (metrics.precision_score(y_test, y_pred)))

Accuracy  is  :  0.7734734077478661
Recall  is    :  0.7439024390243902
Precision  is :  0.7338345864661654
